# SPLIT-APPLY-COMBINE

Pandas has the groupby method that allows you to group rows of a DataFrame by the values of a column and then apply a function to each group. That way we don't need to go across all the rows of the DataFrame to calculate some statistics which might become a slow task. 

## Spliting

In [96]:
import pandas as pd
import numpy as np

df = pd.read_csv("census.csv")

#we will exclude state level summaries, which have a SUMLEV value of 40
df = df[df['SUMLEV'] == 50]

df.head()

,SUMLEV,REGION,DIVISION,STATE,COUNTY,STNAME,CTYNAME,CENSUS2010POP,ESTIMATESBASE2010,POPESTIMATE2010,...,RDOMESTICMIG2011,RDOMESTICMIG2012,RDOMESTICMIG2013,RDOMESTICMIG2014,RDOMESTICMIG2015,RNETMIG2011,RNETMIG2012,RNETMIG2013,RNETMIG2014,RNETMIG2015
1,50,3,6,1,1,Alabama,Autauga County,54571,54571,54660,...,7.242091,-2.915927,-3.012349,2.265971,-2.530799,7.606016,-2.626146,-2.722002,2.592270,-2.187333
2,50,3,6,1,3,Alabama,Baldwin County,182265,182265,183193,...,14.832960,17.647293,21.845705,19.243287,17.197872,15.844176,18.559627,22.727626,20.317142,18.293499
3,50,3,6,1,5,Alabama,Barbour County,27457,27457,27341,...,-4.728132,-2.500690,-7.056824,-3.904217,-10.543299,-4.874741,-2.758113,-7.167664,-3.978583,-10.543299
4,50,3,6,1,7,Alabama,Bibb County,22915,22919,22861,...,-5.527043,-5.068871,-6.201001,-0.177537,0.177258,-5.088389,-4.363636,-5.403729,0.754533,1.107861
5,50,3,6,1,9,Alabama,Blount County,57322,57322,57373,...,1.807375,-1.177622,-1.748766,-2.062535,-1.369970,1.859511,-0.848580,-1.402476,-1.577232,-0.884411


## average population of counties per state
1st method

In [97]:
%%timeit -n 3

for state in df['STNAME'].unique():
    
    avg = np.average(df.where(df['STNAME'] == state).dropna()['CENSUS2010POP']) 
    print('Counties in state ' + state + ' have an average population of ' + str(avg))


Counties in state Alabama have an average population of 71339.34328358209
Counties in state Alaska have an average population of 24490.724137931036
Counties in state Arizona have an average population of 426134.4666666667
Counties in state Arkansas have an average population of 38878.90666666667
Counties in state California have an average population of 642309.5862068966
Counties in state Colorado have an average population of 78581.1875
Counties in state Connecticut have an average population of 446762.125
Counties in state Delaware have an average population of 299311.3333333333
Counties in state District of Columbia have an average population of 601723.0
Counties in state Florida have an average population of 280616.5671641791
Counties in state Georgia have an average population of 60928.63522012578
Counties in state Hawaii have an average population of 272060.2
Counties in state Idaho have an average population of 35626.86363636364
Counties in state Illinois have an average populat

2nd method using groupby()

In [98]:
df.groupby("STNAME")

In [99]:
%%timeit -n 10

# 1) SPLIT: we tell pandas we are interested in grouping by state name
for group, frame in df.groupby('STNAME'): #we have two values because groupby returns a tuple with the key we are grouping on and the projected dataframe
    
    # 2) APPLY: the logic. We apply a function to each group

    avg = np.average(frame['CENSUS2010POP'])

    print('Counties in state ' + group + ' have an average population of ' + str(avg))

Counties in state Alabama have an average population of 71339.34328358209
Counties in state Alaska have an average population of 24490.724137931036
Counties in state Arizona have an average population of 426134.4666666667
Counties in state Arkansas have an average population of 38878.90666666667
Counties in state California have an average population of 642309.5862068966
Counties in state Colorado have an average population of 78581.1875
Counties in state Connecticut have an average population of 446762.125
Counties in state Delaware have an average population of 299311.3333333333
Counties in state District of Columbia have an average population of 601723.0
Counties in state Florida have an average population of 280616.5671641791
Counties in state Georgia have an average population of 60928.63522012578
Counties in state Hawaii have an average population of 272060.2
Counties in state Idaho have an average population of 35626.86363636364
Counties in state Illinois have an average populat

## Using a function to split our data

We'll create a function that called set_batch_number and if the first letter if the parameter is a capital M we'll return 0. If it's a capital Q we'll return 1. Otherwise we'll return 2. Then will pass this function to the groupby method. We need to set the index to the column we want to group by. In this case, the state column. Then we'll pass the function to the groupby method. The groupby method will call the function for each value of the state column and will group the rows by the value returned by the function.

In [100]:
df = df.set_index('STNAME')

def set_batch_number(item):
    if item[0] < 'M':
        return 0
    if item[0] < 'Q':
        return 1
    return 2

for group, frame in df.groupby(set_batch_number):  #if no column is passed, the groupby function will use the index. Function applied to the index
    print('There are ' + str(len(frame)) + ' records in group ' + str(group) + ' for processing.')

There are 1177 records in group 0 for processing.
There are 1134 records in group 1 for processing.
There are 831 records in group 2 for processing.


## Another example

There are two columns of interest: cancelation_policy and review_scores value.

In [101]:
df = pd.read_csv("listings.csv")

df.head()

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,...,review_scores_value,requires_license,license,jurisdiction_names,instant_bookable,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month
0,12147973,https://www.airbnb.com/rooms/12147973,20160906204935,2016-09-07,Sunny Bungalow in the City,"Cozy, sunny, family home. Master bedroom high...",The house has an open and cozy feel at the sam...,"Cozy, sunny, family home. Master bedroom high...",none,"Roslindale is quiet, convenient and friendly. ...",...,NaN,f,NaN,NaN,f,moderate,f,f,1,NaN
1,3075044,https://www.airbnb.com/rooms/3075044,20160906204935,2016-09-07,Charming room in pet friendly apt,Charming and quiet room in a second floor 1910...,Small but cozy and quite room with a full size...,Charming and quiet room in a second floor 1910...,none,"The room is in Roslindale, a diverse and prima...",...,9.0,f,NaN,NaN,t,moderate,f,f,1,1.30
2,6976,https://www.airbnb.com/rooms/6976,20160906204935,2016-09-07,Mexican Folk Art Haven in Boston,"Come stay with a friendly, middle-aged guy in ...","Come stay with a friendly, middle-aged guy in ...","Come stay with a friendly, middle-aged guy in ...",none,The LOCATION: Roslindale is a safe and diverse...,...,10.0,f,NaN,NaN,f,moderate,t,f,1,0.47
3,1436513,https://www.airbnb.com/rooms/1436513,20160906204935,2016-09-07,Spacious Sunny Bedroom Suite in Historic Home,Come experience the comforts of home away from...,Most places you find in Boston are small howev...,Come experience the comforts of home away from...,none,Roslindale is a lovely little neighborhood loc...,...,10.0,f,NaN,NaN,f,moderate,f,f,1,1.00
4,7651065,https://www.airbnb.com/rooms/7651065,20160906204935,2016-09-07,Come Home to Boston,"My comfy, clean and relaxing home is one block...","Clean, attractive, private room, one block fro...","My comfy, clean and relaxing home is one block...",none,"I love the proximity to downtown, the neighbor...",...,10.0,f,NaN,NaN,f,flexible,f,f,1,2.25


In [102]:
# Grouping by both of these columns. 1st we can promote the columns to a multi-level index and call groupby

df = df.set_index(['cancellation_policy', 'review_scores_value'])

for group, frame in df.groupby(level=(0,1)):
    print(group)

('flexible', 2.0)
('flexible', 4.0)
('flexible', 5.0)
('flexible', 6.0)
('flexible', 7.0)
('flexible', 8.0)
('flexible', 9.0)
('flexible', 10.0)
('moderate', 2.0)
('moderate', 4.0)
('moderate', 6.0)
('moderate', 7.0)
('moderate', 8.0)
('moderate', 9.0)
('moderate', 10.0)
('strict', 2.0)
('strict', 3.0)
('strict', 4.0)
('strict', 5.0)
('strict', 6.0)
('strict', 7.0)
('strict', 8.0)
('strict', 9.0)
('strict', 10.0)
('super_strict_30', 6.0)
('super_strict_30', 7.0)
('super_strict_30', 8.0)
('super_strict_30', 9.0)
('super_strict_30', 10.0)


In [103]:
df

id  \
cancellation_policy review_scores_value             
moderate            NaN                  12147973   
                    9.0                   3075044   
                    10.0                     6976   
                    10.0                  1436513   
flexible            10.0                  7651065   
...                                           ...   
strict              9.0                   8373729   
                    NaN                  14844274   
flexible            NaN                  14585486   
strict              7.0                  14603878   
flexible            NaN                  14504422   

                                                                   listing_url  \
cancellation_policy review_scores_value                                          
moderate            NaN                  https://www.airbnb.com/rooms/12147973   
                    9.0                   https://www.airbnb.com/rooms/3075044   
                    10.0                     https://www.airbnb.com/rooms/6976   
                    10.0                  https://www.airbnb.com/rooms/1436513   
flexible            10.0                  https://www.airbnb.com/rooms/7651065   
...                                                                        ...   
strict              9.0                   https://www.airbnb.com/rooms/8373729   
                    NaN                  https://www.airbnb.com/rooms/14844274   
flexible            NaN                  https://www.airbnb.com/rooms/14585486   
strict              7.0                  https://www.airbnb.com/rooms/14603878   
flexible            NaN                  https://www.airbnb.com/rooms/14504422   

                                              scrape_id last_scraped  \
cancellation_policy review_scores_value                                
moderate            NaN                  20160906204935   2016-09-07   
                    9.0                  20160906204935   2016-09-07   
                    10.0                 20160906204935   2016-09-07   
                    10.0                 20160906204935   2016-09-07   
flexible            10.0                 20160906204935   2016-09-07   
...                                                 ...          ...   
strict              9.0                  20160906204935   2016-09-07   
                    NaN                  20160906204935   2016-09-07   
flexible            NaN                  20160906204935   2016-09-07   
strict              7.0                  20160906204935   2016-09-07   
flexible            NaN                  20160906204935   2016-09-07   

                                                                                  name  \
cancellation_policy review_scores_value                                                  
moderate            NaN                                     Sunny Bungalow in the City   
                    9.0                              Charming room in pet friendly apt   
                    10.0                              Mexican Folk Art Haven in Boston   
                    10.0                 Spacious Sunny Bedroom Suite in Historic Home   
flexible            10.0                                           Come Home to Boston   
...                                                                                ...   
strict              9.0                                           Big cozy room near T   
                    NaN                            BU Apartment DexterPark Bright room   
flexible            NaN                                       Gorgeous funky apartment   
strict              7.0                          Great Location; Train and Restaurants   
flexible            NaN                             (K1) Private Room near Harvard/MIT   

                                                                                   summary  \
cancellation_policy review_scores_value                                                 

What if we want to separate the 10s from the rest of the revhew_score values?

In [104]:
def grouping_fun(item):
    # item is in the format of (cancellation_policy, review_scores_value)
    if item[1] == 10:
        return (item[0], '10')
    else:
        return (item[0], 'not 10')
    

for group, frame in df.groupby(by=grouping_fun):
    print(group)

('flexible', '10')
('flexible', 'not 10')
('moderate', '10')
('moderate', 'not 10')
('strict', '10')
('strict', 'not 10')
('super_strict_30', '10')
('super_strict_30', 'not 10')


In [105]:
df

id  \
cancellation_policy review_scores_value             
moderate            NaN                  12147973   
                    9.0                   3075044   
                    10.0                     6976   
                    10.0                  1436513   
flexible            10.0                  7651065   
...                                           ...   
strict              9.0                   8373729   
                    NaN                  14844274   
flexible            NaN                  14585486   
strict              7.0                  14603878   
flexible            NaN                  14504422   

                                                                   listing_url  \
cancellation_policy review_scores_value                                          
moderate            NaN                  https://www.airbnb.com/rooms/12147973   
                    9.0                   https://www.airbnb.com/rooms/3075044   
                    10.0                     https://www.airbnb.com/rooms/6976   
                    10.0                  https://www.airbnb.com/rooms/1436513   
flexible            10.0                  https://www.airbnb.com/rooms/7651065   
...                                                                        ...   
strict              9.0                   https://www.airbnb.com/rooms/8373729   
                    NaN                  https://www.airbnb.com/rooms/14844274   
flexible            NaN                  https://www.airbnb.com/rooms/14585486   
strict              7.0                  https://www.airbnb.com/rooms/14603878   
flexible            NaN                  https://www.airbnb.com/rooms/14504422   

                                              scrape_id last_scraped  \
cancellation_policy review_scores_value                                
moderate            NaN                  20160906204935   2016-09-07   
                    9.0                  20160906204935   2016-09-07   
                    10.0                 20160906204935   2016-09-07   
                    10.0                 20160906204935   2016-09-07   
flexible            10.0                 20160906204935   2016-09-07   
...                                                 ...          ...   
strict              9.0                  20160906204935   2016-09-07   
                    NaN                  20160906204935   2016-09-07   
flexible            NaN                  20160906204935   2016-09-07   
strict              7.0                  20160906204935   2016-09-07   
flexible            NaN                  20160906204935   2016-09-07   

                                                                                  name  \
cancellation_policy review_scores_value                                                  
moderate            NaN                                     Sunny Bungalow in the City   
                    9.0                              Charming room in pet friendly apt   
                    10.0                              Mexican Folk Art Haven in Boston   
                    10.0                 Spacious Sunny Bedroom Suite in Historic Home   
flexible            10.0                                           Come Home to Boston   
...                                                                                ...   
strict              9.0                                           Big cozy room near T   
                    NaN                            BU Apartment DexterPark Bright room   
flexible            NaN                                       Gorgeous funky apartment   
strict              7.0                          Great Location; Train and Restaurants   
flexible            NaN                             (K1) Private Room near Harvard/MIT   

                                                                                   summary  \
cancellation_policy review_scores_value                                                 

In [106]:
# SIMPLE EXAMPLE

import pandas as pd

# Create a sample dataframe
data = {
    'name': ['Alice', 'Bob', 'Charlie', 'David', 'Eve'],
    'age': [25, 32, 35, 19, 45]
}
df = pd.DataFrame(data)

# Define a function to categorize ages
def age_group(age):
    if age < 30:
        return 'Young'
    else:
        return 'Old'

# Group the dataframe by age category
grouped = df.groupby(df['age'].apply(age_group))

# Now we can perform operations on these groups
print(grouped.mean())

             age
age             
Old    37.333333
Young  22.000000


In [107]:
set

for group, frame in df.groupby(age_group):
    print(group)
    print(frame)

Young
      name  age
0    Alice   25
1      Bob   32
2  Charlie   35
3    David   19
4      Eve   45


## Agregation

As an example, let's get the average review_scores value for each cancelation_policy value.

In [108]:
df = pd.read_csv("listings.csv")

df = df.reset_index()

df.groupby("cancellation_policy").agg({"review_scores_value": np.average})

,review_scores_value
cancellation_policy,
flexible,NaN
moderate,NaN
strict,NaN
super_strict_30,NaN


In [109]:
df.columns

Index(['index', 'id', 'listing_url', 'scrape_id', 'last_scraped', 'name',
       'summary', 'space', 'description', 'experiences_offered',
       'neighborhood_overview', 'notes', 'transit', 'access', 'interaction',
       'house_rules', 'thumbnail_url', 'medium_url', 'picture_url',
       'xl_picture_url', 'host_id', 'host_url', 'host_name', 'host_since',
       'host_location', 'host_about', 'host_response_time',
       'host_response_rate', 'host_acceptance_rate', 'host_is_superhost',
       'host_thumbnail_url', 'host_picture_url', 'host_neighbourhood',
       'host_listings_count', 'host_total_listings_count',
       'host_verifications', 'host_has_profile_pic', 'host_identity_verified',
       'street', 'neighbourhood', 'neighbourhood_cleansed',
       'neighbourhood_group_cleansed', 'city', 'state', 'zipcode', 'market',
       'smart_location', 'country_code', 'country', 'latitude', 'longitude',
       'is_location_exact', 'property_type', 'room_type', 'accommodates',
       'ba

In [110]:
df.groupby("cancellation_policy").agg({"review_scores_value": np.nanmean}) #np.average doesn't ignore NaN values

,review_scores_value
cancellation_policy,
flexible,9.237421
moderate,9.307398
strict,9.081441
super_strict_30,8.537313


We can extend the dictionary to aggregate by multiple functions or columns.

In [111]:
df.groupby("cancellation_policy").agg({"review_scores_value":[np.nanmean, np.nanstd] , "reviews_per_month": np.nanmean}) #nanstd is the standard deviation ignoring NaN values

review_scores_value           reviews_per_month
                                nanmean    nanstd           nanmean
cancellation_policy                                                
flexible                       9.237421  1.096271          1.829210
moderate                       9.307398  0.859859          2.391922
strict                         9.081441  1.040531          1.873467
super_strict_30                8.537313  0.840785          0.340143

## Transformation

While aggregation must return a reduced version of the data, transformation can return some transformed version of the full data to recombine. For such a transformation, the output is the same shape as the input. A common use-case is to center the data by subtracting the group-wise mean.

1) We'll use the transform method to calculate the mean of the group for each listing in the Dataframe

In [112]:
cols = ['cancellation_policy', 'review_scores_value']

transform_df = df[cols].groupby('cancellation_policy').transform(np.nanmean)
transform_df.head()

,review_scores_value
0,9.307398
1,9.307398
2,9.307398
3,9.307398
4,9.237421


In [113]:
# Above we got the results in a column with the same lenght and name as ib the original Dataframe. We can change the name of the column before merging it with the original Dataframe.

transform_df.rename({'review_scores_value': 'mean_review_scores'}, axis='columns', inplace=True)

In [114]:
df = df.merge(transform_df, left_index=True, right_index=True)
df.head()

,index,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,...,requires_license,license,jurisdiction_names,instant_bookable,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month,mean_review_scores
0,0,12147973,https://www.airbnb.com/rooms/12147973,20160906204935,2016-09-07,Sunny Bungalow in the City,"Cozy, sunny, family home. Master bedroom high...",The house has an open and cozy feel at the sam...,"Cozy, sunny, family home. Master bedroom high...",none,...,f,NaN,NaN,f,moderate,f,f,1,NaN,9.307398
1,1,3075044,https://www.airbnb.com/rooms/3075044,20160906204935,2016-09-07,Charming room in pet friendly apt,Charming and quiet room in a second floor 1910...,Small but cozy and quite room with a full size...,Charming and quiet room in a second floor 1910...,none,...,f,NaN,NaN,t,moderate,f,f,1,1.30,9.307398
2,2,6976,https://www.airbnb.com/rooms/6976,20160906204935,2016-09-07,Mexican Folk Art Haven in Boston,"Come stay with a friendly, middle-aged guy in ...","Come stay with a friendly, middle-aged guy in ...","Come stay with a friendly, middle-aged guy in ...",none,...,f,NaN,NaN,f,moderate,t,f,1,0.47,9.307398
3,3,1436513,https://www.airbnb.com/rooms/1436513,20160906204935,2016-09-07,Spacious Sunny Bedroom Suite in Historic Home,Come experience the comforts of home away from...,Most places you find in Boston are small howev...,Come experience the comforts of home away from...,none,...,f,NaN,NaN,f,moderate,f,f,1,1.00,9.307398
4,4,7651065,https://www.airbnb.com/rooms/7651065,20160906204935,2016-09-07,Come Home to Boston,"My comfy, clean and relaxing home is one block...","Clean, attractive, private room, one block fro...","My comfy, clean and relaxing home is one block...",none,...,f,NaN,NaN,f,flexible,f,f,1,2.25,9.237421


2) Difference between a given row and it's group mean

In [115]:
df["mean_diff"] = np.absolute(df["review_scores_value"] - df["mean_review_scores"])
df["mean_diff"].head()

0         NaN
1    0.307398
2    0.692602
3    0.692602
4    0.762579
Name: mean_diff, dtype: float64

## Filtering
For instance, suppose that we want to filter out rows belonging to a group with average review_scores value less than 9.

In [116]:
df.groupby("cancellation_policy").filter(lambda x: np.nanmean(x["review_scores_value"]) > 9.2) 

#it should return only moderate cancellation policy

,index,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,...,license,jurisdiction_names,instant_bookable,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month,mean_review_scores,mean_diff
0,0,12147973,https://www.airbnb.com/rooms/12147973,20160906204935,2016-09-07,Sunny Bungalow in the City,"Cozy, sunny, family home. Master bedroom high...",The house has an open and cozy feel at the sam...,"Cozy, sunny, family home. Master bedroom high...",none,...,NaN,NaN,f,moderate,f,f,1,NaN,9.307398,NaN
1,1,3075044,https://www.airbnb.com/rooms/3075044,20160906204935,2016-09-07,Charming room in pet friendly apt,Charming and quiet room in a second floor 1910...,Small but cozy and quite room with a full size...,Charming and quiet room in a second floor 1910...,none,...,NaN,NaN,t,moderate,f,f,1,1.30,9.307398,0.307398
2,2,6976,https://www.airbnb.com/rooms/6976,20160906204935,2016-09-07,Mexican Folk Art Haven in Boston,"Come stay with a friendly, middle-aged guy in ...","Come stay with a friendly, middle-aged guy in ...","Come stay with a friendly, middle-aged guy in ...",none,...,NaN,NaN,f,moderate,t,f,1,0.47,9.307398,0.692602
3,3,1436513,https://www.airbnb.com/rooms/1436513,20160906204935,2016-09-07,Spacious Sunny Bedroom Suite in Historic Home,Come experience the comforts of home away from...,Most places you find in Boston are small howev...,Come experience the comforts of home away from...,none,...,NaN,NaN,f,moderate,f,f,1,1.00,9.307398,0.692602
4,4,7651065,https://www.airbnb.com/rooms/7651065,20160906204935,2016-09-07,Come Home to Boston,"My comfy, clean and relaxing home is one block...","Clean, attractive, private room, one block fro...","My comfy, clean and relaxing home is one block...",none,...,NaN,NaN,f,flexible,f,f,1,2.25,9.237421,0.762579
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3576,3576,14689681,https://www.airbnb.com/rooms/14689681,20160906204935,2016-09-07,Beautiful loft style bedroom with large bathroom,You'd be living on the top floor of a four sto...,NaN,You'd be living on the top floor of a four sto...,none,...,NaN,NaN,f,flexible,f,f,1,NaN,9.237421,NaN
3577,3577,13750763,https://www.airbnb.com/rooms/13750763,20160906204935,2016-09-07,Comfortable Space in the Heart of Brookline,"Our place is close to Coolidge Corner, Allston...",This space consists of 2 Rooms and a private b...,"Our place is close to Coolidge Corner, Allston...",none,...,NaN,NaN,f,flexible,f,f,1,NaN,9.237421,NaN
3579,3579,14852179,https://www.airbnb.com/rooms/14852179,20160906204935,2016-09-07,Spacious Queen Bed Room Close to Boston Univer...,- Grocery: A full-size Star market is 2 minute...,NaN,- Grocery: A full-size Star market is 2 minute...,none,...,NaN,NaN,f,flexible,f,f,1,NaN,9.237421,NaN
3582,3582,14585486,https://www.airbnb.com/rooms/14585486,20160906204935,2016-09-07,Gorgeous funky apartment,Funky little apartment close to public transpo...,Modern and relaxed space with many facilities ...,Funky little apartment close to public transpo...,none,...,NaN,NaN,f,flexible,f,f,1,NaN,9.237421,NaN


## Apply

In a previous example we wanted to find the average review_scores value for each cancelation_policy value and the deviation of a listing from the group mean. We used a two step process: first we used transform() and they we broadcasted to create a new column. With apply() we can do both at once.


In [117]:
df = pd.read_csv("listings.csv")

df = df[['cancellation_policy', 'review_scores_value']]
df.head()

,cancellation_policy,review_scores_value
0,moderate,NaN
1,moderate,9.0
2,moderate,10.0
3,moderate,10.0
4,flexible,10.0


In [118]:
def calc_mean_review_scores(group):
    avg = np.nanmean(group["review_scores_value"])
    group["review_scores_value"] = np.abs(avg - group["review_scores_value"])
    return group

df.groupby("cancellation_policy").apply(calc_mean_review_scores).head()

,cancellation_policy,review_scores_value
0,moderate,NaN
1,moderate,0.307398
2,moderate,0.692602
3,moderate,0.692602
4,flexible,0.762579
